In [1]:
# required library: modelx
# pip install modelx
import modelx as mx
import numpy as np
import pandas as pd

In [2]:
model = mx.new_model("TermAssurance")
space = model.new_space("Projection")

In [3]:
model.np = np

In [4]:
df = pd.read_csv("qx_non_smoker_generic.csv", index_col=0)
df.columns = [i for i in "012345"]
df.index.name = "Age"

In [5]:
space.new_pandas("mort_qx_smoker", "Projection/mortality.csv", df, filetype="csv")
space.mort_qx_non_smoker = space.mort_qx_smoker

In [6]:
series = pd.read_csv("uk_zero_spot.csv", index_col=0, squeeze=True)

In [7]:
space.new_pandas("disc_rate_annual", "Projection/disc_rate_annual.csv", series, filetype="csv")

year
0      0.00000
1      0.00555
2      0.00684
3      0.00788
4      0.00866
        ...   
146    0.03025
147    0.03033
148    0.03041
149    0.03049
150    0.03056
Name: zero_spot, Length: 151, dtype: float64

In [8]:
mp = pd.read_csv("data/term_data_1k.csv", index_col=0)
space.new_pandas("model_point_data", "Projection/model_points.csv", mp, filetype="csv")

,sum_assured,age_at_entry,term_y,smoker_status,shape,annual_premium,init_pols_if,extra_mortality,sex
point_id,,,,,,,,,
1,861683.5069,39,20,S,level,1,1,0,M
2,568669.7775,53,10,N,decreasing,1,1,0,F
3,832762.1352,46,15,N,level,1,1,0,M
4,718116.2083,30,20,S,level,1,1,0,F
5,987991.6808,23,15,N,decreasing,1,1,0,M
...,...,...,...,...,...,...,...,...,...
996,766862.7945,33,10,N,level,1,1,0,F
997,673576.2369,57,10,S,level,1,1,0,F
998,671508.7888,39,10,N,decreasing,1,1,0,M


In [9]:
space.point_id = 1

In [10]:
@mx.defcells
def model_point():
    return model_point_data.loc[point_id]

@mx.defcells
def sum_assured(): return model_point()["sum_assured"]
    
@mx.defcells
def age_at_entry(): return model_point()["age_at_entry"]
    
@mx.defcells
def term_y(): return model_point()["term_y"]
    
@mx.defcells
def smoker_status(): return model_point()["smoker_status"]

@mx.defcells    
def shape(): return model_point()["shape"]
    
@mx.defcells
def annual_premium(): return model_point()["annual_premium"]

@mx.defcells    
def init_pols_if(): return 1

@mx.defcells    
def extra_mortality(): return 1 if smoker_status() == "S" else 0

@mx.defcells    
def sex(): return model_point()["sex"]

@mx.defcells
def proj_len(): return 12 * term_y() + 1

@mx.defcells
def cost_inflation_pa(): return 0.02

@mx.defcells    
def initial_expense(): return 500

@mx.defcells    
def expense_pp(): return 10

@mx.defcells
def lapse_rate_pa(): return 0.1

@mx.defcells
def disc_rate_monthly():
    return np.array(list((1 + disc_rate_annual[t//12])**(1/12) - 1 for t in range(proj_len())))

@mx.defcells
def disc_factors():
    return np.array(list((1 + disc_rate_monthly()[t])**(-t) for t in range(proj_len())))


In [11]:
@mx.defcells
def net_cf(t):
    return premiums(t) - claims(t) - expenses(t)

@mx.defcells
def premium_pp(t):
    """monthly premium"""
    return annual_premium / 12

@mx.defcells
def claim_pp(t):
    if t == 0:
        return sum_assured()
    elif t > term_y() * 12:
        return 0
    elif shape == "level":
        return sum_assured()
    elif shape == "decreasing":
        r = (1+0.07)**(1/12)-1
        S = sum_assured()
        T = term_y() * 12
        outstanding = S * ((1+r)**T - (1+r)**t)/((1+r)**T - 1)
        return outstanding
    else:
        raise ValueError("Parameter 'shape' must be 'level' or 'decreasing'")

@mx.defcells
def inflation_factor(t):
    """annual"""
    return (1 + cost_inflation_pa)**(t//12)
  
@mx.defcells
def premiums(t):
    return premium_pp(t) * num_pols_if(t)

@mx.defcells
def duration(t):
    """duration in force in years"""
    return t//12

@mx.defcells
def claims(t):
    return claim_pp(t) * num_deaths(t)
  
@mx.defcells
def expenses(t):
    if t == 0:
        return initial_expense()
    else:
        return num_pols_if(t) * expense_pp()/12 * inflation_factor(t)
  
@mx.defcells
def num_pols_if(t):
    """number of policies in force"""
    if t==0:
        return init_pols_if()
    elif t > term_y() * 12:
        return 0
    else:
        return num_pols_if(t-1) - num_exits(t-1) - num_deaths(t-1)

  
@mx.defcells
def num_exits(t):
    """exits occurring at time t"""
    return num_pols_if(t) * (1-(1 - lapse_rate_pa())**(1/12))
  
  
@mx.defcells
def num_deaths(t):
    """deaths occurring at time t"""
    return num_pols_if(t) * q_x_12(t)

@mx.defcells
def age(t):
    return age_at_entry + t//12


@mx.defcells
def q_x_12(t):
    return 1-(1- q_x_rated(t))**(1/12)

@mx.defcells
def qx_non_smoker(t):
    """non-smoker mortality"""
    return mort_qx_non_smoker[str(min(5, duration(t)))][age(t)]

@mx.defcells
def qx_smoker(t):
    """smoker mortality"""
    return mort_qx_smoker[str(min(5, duration(t)))][age(t)]

@mx.defcells
def q_x(t):
    if smoker_status == "N":
        return qx_non_smoker(t)
    elif smoker_status == "S":
        return qx_smoker(t)
        
@mx.defcells
def q_x_rated(t):
    return max(0, min(1 , q_x(t) * (1 + extra_mortality()) ) )


@mx.defcells
def commission(t): return 0


In [12]:
@mx.defcells
def npv_claims():
    return sum(list(claims(t) for t in range(proj_len())) * disc_factors()[:proj_len()])

@mx.defcells
def npv_expenses():
    return sum(list(expenses(t) for t in range(proj_len())) * disc_factors()[:proj_len()])

@mx.defcells
def npv_commission():
    return sum(list(commission(t) for t in range(proj_len())) * disc_factors()[:proj_len()])

@mx.defcells
def npv_premiums():
    return sum(list(premiums(t) for t in range(proj_len())) * disc_factors()[:proj_len()])

@mx.defcells
def annual_risk_premium():
    return (npv_claims() + npv_expenses() + npv_commission()) / npv_premiums()

@mx.defcells
def monthly_risk_premium():
    return round(annual_risk_premium() / 12, 2)

In [13]:
space.formula = lambda point_id: None

In [14]:
monthly_risk_premium()

136.65

In [18]:
sum(list(space[i].monthly_risk_premium() for i in range(1, 101)))

6982.4900000000025

In [15]:
model.zip("TermAssurance.zip")